In [62]:
# This is the code to parse the .spdx file for the SBOMs
# It creates 3 .csv files
# spdx_output.csv: Files independent of packages
# relationships_output.csv: Relationship between differentr packages
# packages_output.csv: Contains different packages

#works from 1.15 to recent

import csv
import glob
import pandas as pd
import linecache

# Define a variable to track the current entry

def cleanup(unwantedTerms,line):
  for i in unwantedTerms:
    line = line.replace(i,"")
  return line


In [89]:
#function to fill in the lists that will later be used to make our 3 csvs
def fill_lists(spdx_file_path):
  with open(spdx_file_path,'r') as spdx_file:
      #counter to keep track of how many dependencies a version has
      #iterate line by line and use "name" as indicator of
      counter = 0
      for line_num,line in enumerate(spdx_file):
        line = line.strip()
        if line.startswith('"name"'):
          #cleanup symbols we do not need
          line = cleanup(['"name":',',','"','"',' '],line)
          #filter out two exceptions and append
          if (line[0:10] == "kubernetes"): versionName = line
          if (line[0:10] != "kubernetes" and line[0:4] != "SBOM"):
            dependencyName.append(line)
            #getting spdxID
            #print(linecache.getline(spdx_file_path,count))
            spdxID.append(cleanup(['"SPDXID":',',','"','"',' '],linecache.getline(spdx_file_path,line_num)))
            #counter keeping track of total amt of dependencies
            counter+=1
        if line.startswith('"versionInfo"'):
          line = cleanup(['"versionInfo":',',','"','"',' '],line)
          #filter out exceptions
          if (line != ""):dependencyVersion.append(line)
      for count in range(counter):
        versionNameList.append(versionName)

In [95]:
def make_csv_dependencies(file_path):
  #csv path for dependencies

  #writing into csv
  with open(file_path,'w') as csv_file:
    fieldnames = ['dependencyName-Version','dependencyName','dependencyVersion','SPDXID']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    csv_writer.writeheader()
    for count in range(len(dependencyName)):
        csv_writer.writerow({
            'dependencyName-Version': dependencyName[count] + '-' + dependencyVersion[count],
            'dependencyName': dependencyName[count],
            'dependencyVersion': dependencyVersion[count],
            'SPDXID': spdxID[count]
            })

In [96]:
def make_csv_map(file_path):
  #csv path for mapping
  #writing into csv
  with open(file_path,'w') as csv_file:
    fieldnames = ['kubernetesVersion','dependencyName-Version']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    csv_writer.writeheader()
    for count in range(len(dependencyName)):
        csv_writer.writerow({
            'kubernetesVersion': versionNameList[count],
            'dependencyName-Version': dependencyName[count] + '-' + dependencyVersion[count]
            })

In [97]:
def make_csv_kubeversion(file_path):
  #csv path for kube versions
  #writing into csv
  with open(file_path,'w') as csv_file:
    fieldnames = ['kubernetesVersion']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    csv_writer.writeheader()
    for count in range(len(versionNameList)):
      csv_writer.writerow({
          'kubernetesVersion': versionNameList[count]

          })

In [104]:
###RUN THIS TO GENERATE

path = '/content/*.*'
# Create lists to store SPDX information
versionName = None
versionNameList = []
dependencyName = []
dependencyVersion = []
spdxID = []

dependenciescsv_file_path = '/content/drive/MyDrive/EC528Colab/dependencies_entry.csv'
mappingcsv_file_path = '/content/drive/MyDrive/EC528Colab/version_dependencies_mapping.csv'
kubeversioncsv_file_path = '/content/drive/MyDrive/EC528Colab/version_entry.csv'

#iterate through all files in this path
for file in glob.glob(path):
  fill_lists(file)

#make csv using respective functions
make_csv_dependencies(dependenciescsv_file_path)
make_csv_map(mappingcsv_file_path)
make_csv_kubeversion(kubeversioncsv_file_path)



In [105]:
###RUN THIS TO CLEAN

dependenciesDF = pd.read_csv(dependenciescsv_file_path)
mappingDF = pd.read_csv(mappingcsv_file_path)
kubeversionDF = pd.read_csv(kubeversioncsv_file_path)

#function to clean the csv of duplicate rows
dependenciesDF = dependenciesDF.drop_duplicates()
mappingDF = mappingDF.drop_duplicates()
kubeversionDF = kubeversionDF.drop_duplicates()

dependenciesDF.to_csv(dependenciescsv_file_path,encoding='utf-8',index=False)
mappingDF.to_csv(mappingcsv_file_path,encoding='utf-8',index=False)
kubeversionDF.to_csv(kubeversioncsv_file_path,encoding='utf-8',index=False)